In [3]:
# spam_ham_bow_kr.py
# -*- coding: utf-8 -*-
"""
단어 빈도(BoW) 기반 한국어 스팸메일 분류기
- CSV 열: '분류' (값: '스팸메일' 또는 '정상메일'), '메시지'
- 모델: CountVectorizer + Multinomial Naive Bayes (기본), 로지스틱 회귀 선택 가능
"""

import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin

# ---------- 1) 간단 전처리(이모지/URL/숫자/이메일 표준화) ----------
class SimpleCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        # 아주 가벼운 규칙만 사용 (형태소 분석기 없이도 동작)
        self.url = re.compile(r'https?://\S+|www\.\S+')
        self.email = re.compile(r'[\w\.-]+@[\w\.-]+\.\w+')
        self.phone = re.compile(r'\b\d{2,4}[- ]?\d{3,4}[- ]?\d{4}\b')
        self.num = re.compile(r'\d+')
        self.multispace = re.compile(r'\s+')

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        cleaned = []
        for s in X:
            s = str(s)
            s = self.url.sub(' <URL> ', s)
            s = self.email.sub(' <EMAIL> ', s)
            s = self.phone.sub(' <PHONE> ', s)
            s = self.num.sub(' <NUM> ', s)
            # 특수문자 대부분 제거(한글/영문/숫자 토큰만 남기기)
            s = re.sub(r'[^가-힣A-Za-z0-9<>\s]', ' ', s)
            s = self.multispace.sub(' ', s).strip()
            cleaned.append(s)
        return cleaned

# ---------- 2) 데이터 로드 ----------
# 파일 경로를 바꿔서 사용하세요.
CSV_PATH = 'email_first150_KO_only.csv'

df = pd.read_csv(CSV_PATH, encoding='utf-8-sig')  # cp949일 경우 encoding='cp949'로
# 열 이름 정규화
df = df.rename(columns={'분류': 'label', '메시지': 'text'})
df = df[['label', 'text']].dropna()

# 라벨 매핑
label_map = {'스팸메일': 1, '정상메일': 0}
df['y'] = df['label'].map(label_map)

# ---------- 3) 학습/검증 분할 ----------
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['y'], test_size=0.2, random_state=42, stratify=df['y']
)

# ---------- 4) 벡터화 + 분류기 파이프라인 ----------
# 한국어 토큰을 잘 줍기 위한 token_pattern (한글/영문/숫자 시퀀스)
token_pat = r'(?u)[가-힣A-Za-z0-9<]+[가-힣A-Za-z0-9<>]*'

bow_nb = Pipeline([
    ('clean', SimpleCleaner()),
    ('vect', CountVectorizer(
        token_pattern=token_pat,
        ngram_range=(1, 2),         # 유니그램 + 바이그램
        min_df=2,                   # 너무 희귀한 단어 제거(데이터 작으면 1로)
        max_df=0.95                 # 너무 흔한 단어 무시
    )),
    ('clf', MultinomialNB(alpha=0.5))
])

# 참고: 로지스틱 회귀로 바꾸고 싶다면 아래 주석을 해제
# bow_lr = Pipeline([
#     ('clean', SimpleCleaner()),
#     ('vect', CountVectorizer(token_pattern=token_pat, ngram_range=(1, 2), min_df=2, max_df=0.95)),
#     ('clf', LogisticRegression(max_iter=200, C=2.0, n_jobs=None))
# ])

# ---------- 5) 학습 ----------
bow_nb.fit(X_train, y_train)

# ---------- 6) 평가 ----------
pred = bow_nb.predict(X_test)
proba = bow_nb.predict_proba(X_test)[:, 1]

print('\n[정확도]', round(accuracy_score(y_test, pred), 4))
print('\n[분류 리포트]\n', classification_report(y_test, pred, target_names=['정상메일', '스팸메일']))
print('\n[혼동 행렬]\n', confusion_matrix(y_test, pred))

# ---------- 7) 스팸 특징 단어 살펴보기 ----------
vect = bow_nb.named_steps['vect']
clf = bow_nb.named_steps['clf']
feature_names = vect.get_feature_names_out()
# MultinomialNB는 log prob를 이용해 가중치 유사값을 볼 수 있음
import numpy as np
log_prob = clf.feature_log_prob_  # shape = (class, feature)
spam_rank = np.argsort((log_prob[1] - log_prob[0]))[::-1]  # 스팸에 더 기여하는 순
top_k = 20
print('\n[스팸에 강하게 기여한 상위 단어/바이그램]')
for idx in spam_rank[:top_k]:
    print(feature_names[idx])

# ---------- 8) 예측 데모 ----------
samples = [
    "무료 쿠폰 지금 받기! 링크 클릭하세요",           # 스팸
    "내일 12시에 미팅 가능한가요?",                 # 정상
    "080-XXXX 수신거부/광고 안내",                 # 스팸
    "전송 표준요금 수신 1.50",                      # 스팸
    "프로젝트 회의 자료 첨부합니다.",               # 정상
    "지금 투자하면 200% 수익 보장!",               # 스팸
    "오늘 저녁 약속 시간 다시 알려주세요.",         # 정상
    "무료 영화 티켓 받으세요 ▶ 클릭",               # 스팸
    "택배가 도착했습니다. 확인 부탁드립니다.",        # 정상
    "긴급: 계정 보안을 위해 비밀번호 재설정 필요",    # 스팸
]
print('\n[샘플 예측]')
for s in samples:
    p = bow_nb.predict_proba([s])[0, 1]
    print(f'{s}  --> 스팸확률={p:.3f}')

# ---------- 9) 모델 저장(선택) ----------
import joblib
joblib.dump(bow_nb, 'spam_ham_bow_kr.joblib')
print("\n모델 파일 저장: spam_ham_bow_kr.joblib")



[정확도] 0.9

[분류 리포트]
               precision    recall  f1-score   support

        정상메일       0.92      0.96      0.94        25
        스팸메일       0.75      0.60      0.67         5

    accuracy                           0.90        30
   macro avg       0.84      0.78      0.80        30
weighted avg       0.89      0.90      0.90        30


[혼동 행렬]
 [[24  1]
 [ 2  3]]

[스팸에 강하게 기여한 상위 단어/바이그램]
로
<num> 로
무료
p
<num> p
당첨
전화 코드
코드
회신
으로
세
분당 <num>
로 전화
분당
p <num>
긴급
<num> 세
<num> 으로
전송
세 이상

[샘플 예측]
무료 쿠폰 지금 받기! 링크 클릭하세요  --> 스팸확률=0.769
내일 12시에 미팅 가능한가요?  --> 스팸확률=0.471
080-XXXX 수신거부/광고 안내  --> 스팸확률=0.639
전송 표준요금 수신 1.50  --> 스팸확률=1.000
프로젝트 회의 자료 첨부합니다.  --> 스팸확률=0.176
지금 투자하면 200% 수익 보장!  --> 스팸확률=0.548
오늘 저녁 약속 시간 다시 알려주세요.  --> 스팸확률=0.213
무료 영화 티켓 받으세요 ▶ 클릭  --> 스팸확률=0.829
택배가 도착했습니다. 확인 부탁드립니다.  --> 스팸확률=0.176
긴급: 계정 보안을 위해 비밀번호 재설정 필요  --> 스팸확률=0.694

모델 파일 저장: spam_ham_bow_kr.joblib
